In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import inforec
import notebook_utils as nu
from model import AbsoluteBuilder, EventBuilder

DB_DIR = 'data'

In [ ]:
inforec.InfoRecDB.init(DB_DIR)

In [3]:
db = inforec.InfoRecDB.open(DB_DIR)

In [5]:
nu.dump_events(db)

,ID,Title,Before,After,Same
0,ca3bcf47-c503-4795-ae61-e76d2144b642,title1,None,None,None
1,495d6e53-04e9-48b3-bccb-67d03f081a28,Title2,None,None,None
2,209bd96c-74bf-4ff2-87fc-ca97a36558af,title3,[title1],None,None


In [10]:
e1 = EventBuilder("title1").build()
e2 = EventBuilder("Title2").desc("description for title2").build()
e3 = EventBuilder("title3").before(e1.id).build()
db.add_item(e1)
db.add_item(e2)
db.add_item(e3)

In [6]:
d1 = AbsoluteBuilder().date('2020-03-01').build()
e4 = EventBuilder('Event4').same(d1.id).build()
db.add_item(d1)
db.add_item(e4)

In [8]:
nu.dump_events(db)

,ID,Title,Before,After,Same
0,ca3bcf47-c503-4795-ae61-e76d2144b642,title1,None,None,None
1,495d6e53-04e9-48b3-bccb-67d03f081a28,Title2,None,None,None
2,209bd96c-74bf-4ff2-87fc-ca97a36558af,title3,[title1],None,None
3,3e29a3e1-c996-4eb8-be5b-f28e490a8691,Event4,None,None,[2020-03-01]


In [14]:
db.write()

In [9]:
event = EventBuilder("title4").before('209bd96c-74bf-4ff2-87fc-ca97a36558af').after('ca3bcf47-c503-4795-ae61-e76d2144b642').build()
db.add_item(event)
nu.dump_events(db)

,ID,Title,Before,After,Same
0,ca3bcf47-c503-4795-ae61-e76d2144b642,title1,None,None,None
1,495d6e53-04e9-48b3-bccb-67d03f081a28,Title2,None,None,None
2,209bd96c-74bf-4ff2-87fc-ca97a36558af,title3,[title1],None,None
3,3e29a3e1-c996-4eb8-be5b-f28e490a8691,Event4,None,None,[2020-03-01]
4,de5a5990-27c3-4175-8876-dd26c1bb1b7c,title4,[title3],[title1],None


In [13]:
db.has_no_conflict() or db.conflicts() or [[ db.get_event(eid).title for eid in lst ] for lst in db.conflicts()]

[['ca3bcf47-c503-4795-ae61-e76d2144b642',
  'de5a5990-27c3-4175-8876-dd26c1bb1b7c',
  '209bd96c-74bf-4ff2-87fc-ca97a36558af']]

In [15]:
%debug

> /home/ryey/coding/inforec/storage.py(102)get_event()
    100         item = self.get_item(id)
    101         if not isinstance(item, Event):
--> 102             raise RuntimeError("The requested item {} is not an Event, but a {}".format(id, type(item)))
    103         return item
    104 

ipdb> exit
